In [1]:
########CLUSTER FIRST, put into groups, append group label to dataframe then run through Deep Learning to find recommended songs.
######## Can I pluck out the X closest points


# Dependencies
import numpy as np
import pandas as pd

In [2]:
voice = pd.read_csv('features2.csv')
voice.head()

,popularity,danceability,energy,loudness,speechiness,duration_ms,tempo,class,Score,Input
0,94,0.851,0.541,-6.825,0.0505,235545,124.959,0,34.918375,40.0
1,97,0.511,0.566,-7.230,0.2000,239836,83.903,0,29.208650,NaN
2,95,0.680,0.563,-5.843,0.0454,231267,145.028,1,37.809855,NaN
3,91,0.900,0.653,-6.962,0.2890,145603,112.503,0,30.590125,NaN
4,84,0.572,0.385,-6.362,0.0308,215733,95.799,0,28.939735,NaN


## Data Pre-Processing

In [3]:
X = voice.drop(["class","Score","Input"], axis=1)
y = voice["class"]
print(X.shape, y.shape)

(100, 7) (100,)


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from keras.utils import to_categorical

X_train, X_test, y_train, y_test = train_test_split(
    X, y, random_state=20, stratify=y)
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)


# Step 1: Label-encode data set
label_encoder = LabelEncoder()
label_encoder.fit(y_train)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)

# Step 2: Convert encoded labels to one-hot-encoding
y_train_categorical = to_categorical(encoded_y_train)
y_test_categorical = to_categorical(encoded_y_test)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:625: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


# Create a Deep Learning Model

In [5]:
from keras.models import Sequential
from keras.layers import Dense

# Create model and add layers
model = Sequential()
model.add(Dense(units=100, activation='selu', input_dim=7))
model.add(Dense(units=100, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=50, activation='relu'))
model.add(Dense(units=2, activation='softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               800       
_________________________________________________________________
dense_2 (Dense)              (None, 100)               10100     
_________________________________________________________________
dense_3 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_4 (Dense)              (None, 50)                2550      
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 102       
Total params: 18,602
Trainable params: 18,602
Non-trainable params: 0
_________________________________________________________________


In [6]:
# Compile and fit the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train_scaled,
    y_train_categorical,
    epochs=20,
    shuffle=True,
    verbose=1
)

Epoch 1/20
75/75 [==============================] - 0s 6ms/step - loss: 0.7309 - acc: 0.4533
Epoch 2/20
75/75 [==============================] - 0s 134us/step - loss: 0.6053 - acc: 0.7733
Epoch 3/20
75/75 [==============================] - 0s 207us/step - loss: 0.5231 - acc: 0.7600
Epoch 4/20
75/75 [==============================] - 0s 220us/step - loss: 0.4548 - acc: 0.7467
Epoch 5/20
75/75 [==============================] - 0s 120us/step - loss: 0.3937 - acc: 0.8000
Epoch 6/20
75/75 [==============================] - 0s 107us/step - loss: 0.3415 - acc: 0.8800
Epoch 7/20
75/75 [==============================] - 0s 133us/step - loss: 0.2932 - acc: 0.9333
Epoch 8/20
75/75 [==============================] - 0s 120us/step - loss: 0.2542 - acc: 0.9867
Epoch 9/20
75/75 [==============================] - 0s 147us/step - loss: 0.2152 - acc: 0.9733
Epoch 10/20
75/75 [==============================] - 0s 187us/step - loss: 0.1865 - acc: 0.9733
Epoch 11/20
75/75 [==============================] 

## Quantify our Trained Model

In [7]:
model_loss, model_accuracy = model.evaluate(
    X_test_scaled, y_test_categorical, verbose=2)
print(
    f"Normal Neural Network - Loss: {model_loss}, Accuracy: {model_accuracy}")

Normal Neural Network - Loss: 0.09980093687772751, Accuracy: 0.9599999785423279


## Make Predictions

In [8]:
encoded_predictions = model.predict_classes(X_test_scaled[:5])
prediction_labels = label_encoder.inverse_transform(encoded_predictions)

In [9]:
print(f"Predicted classes: {prediction_labels}")
print(f"Actual Labels: {list(y_test[:5])}")

Predicted classes: [0 1 0 1 1]
Actual Labels: [0, 1, 0, 1, 1]
